<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Adehs_US_Energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys
# import the connect library for psycopg2
import psycopg2
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
from io import StringIO
import numpy as np

import matplotlib.pyplot as plt

In [3]:
params_dic = {
    "host"      : "localhost",
    "user"      : "postgres",
    "password"  : "postgres",
    "port"      : "5432"
}

In [4]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(params_dic)

Connecting to the PostgreSQL database...
Connection successful


In [5]:
csv_file = "Resources/organised_Gen.csv"
energy_data = pd.read_csv(csv_file)
energy_data = energy_data.drop(columns=['Unnamed: 0'])

energy_df = energy_data.rename(columns={
    "YEAR": "year", 
    "MONTH": "month",
    "STATE": "state",
    "TYPE OF PRODUCER": "producer",
    "ENERGY SOURCE": "source",
    "GENERATION (Megawatthours)": "generated"})


energy_df['producer'] = energy_df['producer'].apply(lambda x: x.replace(',','/'))
    
energy_df.head(10)

,year,month,state,producer,source,generated
0,2001,1,AK,Total Electric Power Industry,Coal,46903.0
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0
4,2001,1,AK,Total Electric Power Industry,Wind,87.0
5,2001,1,AK,Total Electric Power Industry,Total,590145.0
6,2001,1,AK,Electric Generators/ Electric Utilities,Coal,18410.0
7,2001,1,AK,Electric Generators/ Electric Utilities,Petroleum,64883.0
8,2001,1,AK,Electric Generators/ Electric Utilities,Natural Gas,305277.0
9,2001,1,AK,Electric Generators/ Electric Utilities,Hydroelectric Conventional,104549.0


In [6]:
!pip install psycopg2-binary

In [7]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [8]:
def create_table(cursor):
    try:
        # Dropping table iris if exists
        cursor.execute("DROP TABLE IF EXISTS energy;")
        sql = '''CREATE TABLE energy(
        year INT NOT NULL, 
        month INT NOT NULL, 
        state VARCHAR NOT NULL, 
        producer TEXT NOT NULL,
        source VARCHAR NOT NULL,
        generated FLOAT NOT NULL
        )'''
        # Creating a table
        cursor.execute(sql);
        print("energy table is created successfully...............")  
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

In [9]:
# Define function using copy_from() with StringIO to insert the dataframe
def copy_from_dataFile_StringIO(conn, datafrm, table):
    
  # save dataframe to an in memory buffer
    buffer = StringIO()
    datafrm.to_csv(buffer, header=False, index = False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        print("Data inserted using copy_from_datafile_StringIO() successfully....")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

In [10]:
conn = connect(params_dic)
# We set autocommit=True so every command we execute will produce results immediately.
conn.autocommit = True
cursor = conn.cursor()
create_table(cursor)

Connecting to the PostgreSQL database...
Connection successful
energy table is created successfully...............


In [11]:
copy_from_dataFile_StringIO(conn, energy_df, 'energy')

Data inserted using copy_from_datafile_StringIO() successfully....


In [74]:
conn.autocommit = True
cursor = conn.cursor()
  
sql = '''SELECT AVG(generated) 
            FROM energy 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state = 'US-TOTAL'
                ;'''  
cursor.execute(sql)
results = cursor.fetchall()
print(results)
  
conn.commit()
#conn.close()

[(336671618.26007783,)]


In [22]:
sql = '''SELECT YEAR, (SUM(generated))
            FROM energy 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state = 'US-TOTAL'
                AND YEAR != 2022
            Group By YEAR
                ;'''  
cursor.execute(sql)
results2 = cursor.fetchall()
print(results2)

AttributeError: 'list' object has no attribute 'show'

In [66]:
energy_data.head(20158)

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,2001,1,AK,Total Electric Power Industry,Coal,46903.0
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0
4,2001,1,AK,Total Electric Power Industry,Wind,87.0
...,...,...,...,...,...,...
20153,2001,12,US-TOTAL,Total Electric Power Industry,Geothermal,1190232.0
20154,2001,12,US-TOTAL,Total Electric Power Industry,Other Biomass,1252855.0
20155,2001,12,US-TOTAL,Total Electric Power Industry,Pumped Storage,-622574.0
20156,2001,12,US-TOTAL,Total Electric Power Industry,Other,1051329.0


In [19]:
# Create a temporary view and transform the temporary view to retrieve only the "School Organization" values from the "TYPE" column. 
def transform_data():
    
    conn.autocommit = True
    cursor = conn.cursor()
  
    sql = '''SELECT YEAR, (SUM(generated))
            FROM energy 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state = 'US-TOTAL'
                AND YEAR != 2022
            Group By YEAR
                ;'''  
  
    cursor.execute(sql)
    results = cursor.fetchall()
    print(results)
  
    conn.commit()
  
    return results

In [21]:
transform_data().show()

[(2001, 3736643649.0), (2002, 3858452251.0), (2003, 3883185204.0), (2004, 3970555264.0), (2005, 4055422750.0), (2006, 4064702227.0), (2007, 4156744725.0), (2008, 4119387759.0), (2009, 3950330927.0), (2010, 4125059899.0), (2011, 4100656050.12), (2012, 4047765259.1499996), (2013, 4065964067.45), (2014, 4092934691.12), (2015, 4077600939.0), (2016, 4076674986.0), (2017, 4034270559.0), (2018, 4178277345.0), (2019, 4126882145.0), (2020, 4007018594.0), (2021, 4115540153.0)]


AttributeError: 'list' object has no attribute 'show'

In [ ]:
conn.close()